In [1]:
import numpy as np
import h5py
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
#from resnets_utils import *
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline

import tensorflow as tf
import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

c:\users\g50\appdata\local\programs\python\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def load_dataset_flower():
    train_data_x = h5py.File('dataset/features.h5', "r")
    train_data_y = h5py.File('dataset/labels.h5', "r")

    #print(list(train_data_y.keys()))
    features = np.array(train_data_x["dataset_1"][:])
    labels = np.array(train_data_y["dataset_1"][:])
    #print(train_set_x_orig.shape)
    #print(train_set_y_orig.shape)
    
   
    return features,labels

In [14]:
import os
import glob
from shutil import copyfile
class_names = ["daffodil", "snowdrop", "lily_valley", "bluebell", "crocus",
              "iris", "tigerlily", "tulip", "fritillary", "sunflower", 
              "daisy", "colts_foot", "dandelion", "cowslip", "buttercup",
              "windflower", "pansy"]

In [15]:
class_limit = 17
label = 0
#X_train_orig = np.array([]).reshape((0,224,224,3))
#Y_train_orig = np.array([]).reshape((1,0))
X_train_orig = np.array([]).reshape((0,224,224,3))
#Y_train_orig = np.array([]).reshape((1,0))
Y_train_orig = []
#print(X_train_orig.shape)
train_path = "dataset/train"
for x in range(class_limit):
    curr_path = train_path + "\\" + class_names[label]
    os.chdir(curr_path)
    for image_path in glob.glob(curr_path + "\\*.jpg"):
        img = image.load_img(image_path,target_size = (224,224,3))
        x = image.img_to_array(img)
        x = np.expand_dims(x,axis = 0)
        x = preprocess_input(x)
        X_train_orig = np.vstack((X_train_orig,x))
        #print(X_train_orig.shape)
        Y_train_orig.append(label)
    print("Folder with label "+str(class_names[label])+" is complete")
    label+=1

Y_train_orig = np.reshape(Y_train_orig,[1,len(Y_train_orig)])

Folder with label daffodil is complete
Folder with label snowdrop is complete
Folder with label lily_valley is complete
Folder with label bluebell is complete
Folder with label crocus is complete
Folder with label iris is complete
Folder with label tigerlily is complete
Folder with label tulip is complete
Folder with label fritillary is complete
Folder with label sunflower is complete
Folder with label daisy is complete
Folder with label colts_foot is complete
Folder with label dandelion is complete
Folder with label cowslip is complete
Folder with label buttercup is complete
Folder with label windflower is complete
Folder with label pansy is complete


In [16]:
print(X_train_orig.shape)
print(Y_train_orig.shape)

(1190, 224, 224, 3)
(1, 1190)


In [18]:
class_limit = 17
label = 0
X_test_orig = np.array([]).reshape((0,224,224,3))
Y_test_orig = []

test_path = "dataset/validation"
for x in range(class_limit):
    curr_path = test_path + "\\" + class_names[label]
    os.chdir(curr_path)
    for image_path in glob.glob(curr_path + "\\*.jpg"):
        img = image.load_img(image_path,target_size = (224,224,3))
        x = image.img_to_array(img)
        x = np.expand_dims(x,axis = 0)
        x = preprocess_input(x)
        X_test_orig = np.vstack((X_test_orig,x))
        #print(X_train_orig.shape)
        Y_test_orig.append(label)
    print("Folder with label "+str(class_names[label])+" is complete")
    label+=1

Y_test_orig = np.reshape(Y_test_orig,[1,len(Y_test_orig)])

Folder with label daffodil is complete
Folder with label snowdrop is complete
Folder with label lily_valley is complete
Folder with label bluebell is complete
Folder with label crocus is complete
Folder with label iris is complete
Folder with label tigerlily is complete
Folder with label tulip is complete
Folder with label fritillary is complete
Folder with label sunflower is complete
Folder with label daisy is complete
Folder with label colts_foot is complete
Folder with label dandelion is complete
Folder with label cowslip is complete
Folder with label buttercup is complete
Folder with label windflower is complete
Folder with label pansy is complete


In [19]:
print(X_test_orig.shape)
print(Y_test_orig.shape)

(170, 224, 224, 3)
(1, 170)


In [20]:
def convert_to_one_hot(A,C):
    A = np.eye(C)[A.reshape(-1)].T
    return A

In [21]:
X_test = X_test_orig/255.

Y_test = convert_to_one_hot(Y_test_orig, 17).T
print ("Y_test shape: " + str(Y_test.shape))

Y_test shape: (170, 17)


In [3]:
features,labels = load_dataset_flower()

In [4]:
print(features.shape)
labels = np.reshape(labels,[labels.shape[0],1])
print(labels.shape)

(1360, 2048)
(1360, 1)


In [5]:
from sklearn.model_selection import train_test_split
test_size = 170
seed = 1
(trainData, testData, trainLabels, testLabels) = train_test_split(features,
                                                                  labels,
                                                                  test_size=test_size,
                                                                  random_state=seed)

In [6]:
print(trainData.shape)
print(testData.shape)
print(trainLabels.shape)
print(testLabels.shape)

(1190, 2048)
(170, 2048)
(1190, 1)
(170, 1)


In [7]:
from sklearn.linear_model import LogisticRegression

#trainLabels = column_or_1d(trainLabels, warn=True)

model = LogisticRegression(random_state=seed)
model.fit(trainData, trainLabels)

c:\users\g50\appdata\local\programs\python\python36\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=1, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [8]:
rank_1 = 0
rank_5 = 0

# loop over test data
for (label, features) in zip(testLabels, testData):
    # predict the probability of each class label and
    # take the top-5 class labels
    predictions = model.predict_proba(np.atleast_2d(features))[0]
    predictions = np.argsort(predictions)[::-1][:5]
    
    # rank-1 prediction increment
    if label == predictions[0]:
        rank_1 += 1

    #rank-5 prediction increment
    if label in predictions:
        rank_5 += 1

# convert accuracies to percentages
rank_1 = (rank_1 / float(len(testLabels))) * 100
rank_5 = (rank_5 / float(len(testLabels))) * 100
print(rank_1)
print(rank_5)

58.82352941176471
92.3529411764706


In [9]:
from sklearn.metrics import classification_report
preds = model.predict(testData)
print(format(classification_report(testLabels, preds)))

             precision    recall  f1-score   support

          0       0.17      0.14      0.15         7
          1       0.69      0.75      0.72        12
          2       0.50      0.77      0.61        13
          3       0.44      0.31      0.36        13
          4       0.25      0.29      0.27         7
          5       0.50      0.38      0.43         8
          6       1.00      0.80      0.89         5
          7       0.62      0.42      0.50        12
          8       0.46      0.67      0.55         9
          9       0.73      0.80      0.76        10
         10       0.86      0.67      0.75         9
         11       0.33      0.33      0.33         6
         12       0.65      0.79      0.71        14
         13       0.83      0.71      0.77        14
         14       0.78      0.58      0.67        12
         15       0.30      0.33      0.32         9
         16       0.82      0.90      0.86        10

avg / total       0.60      0.59      0.59  